Correlation based Feature Selection Algorithm

In [2]:
import pandas as pd

df = pd.read_excel("dataset.xlsx", index_col=0, engine="openpyxl")
df.head(3)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X14,X15,X16,X17,X18,X19,X20,X21,Y,Delta Y
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-01,0.000000,0.0,0.00000,0,1.000000,1.000000,0.073930,0.0,0.0,0,...,0.732353,1.000000,0.516234,0.487288,0.680123,0.984565,0.802498,0.440834,1.000000,0.000000
2020-02-01,0.000000,0.0,0.00000,0,1.000000,1.000000,0.073930,0.0,0.0,0,...,0.732353,1.000000,0.516234,0.487288,0.680123,0.984565,0.802498,0.440834,1.000000,0.000000
2020-03-01,0.001556,0.0,0.11039,1,0.074844,0.881306,0.076524,0.0,0.0,0,...,0.620588,0.895928,0.516234,0.494350,0.680123,0.724255,0.804665,0.202206,1.001898,0.001898


In [3]:
import numpy as np

label = 'Delta Y'
features = df.columns.tolist()
features.remove('Y')
features.remove(label)

In [4]:
from scipy.stats import pointbiserialr
from math import sqrt

def getMerit(subset, label):
    k = len(subset)

    # Average feature-class correlation
    rcf_all = []
    for feature in subset:
        coeff = pointbiserialr( df[label], df[feature] )
        rcf_all.append( abs( coeff.correlation ) )
    rcf = np.mean( rcf_all )

    # Average feature-feature correlation
    corr = df[subset].corr()
    corr.values[np.tril_indices_from(corr.values)] = np.nan
    corr = abs(corr)
    rff = corr.unstack().mean()
    return (k * rcf) / sqrt(k + k * (k-1) * rff)

In [5]:
best_value = -1
best_feature = ''
for feature in features:
    coeff = pointbiserialr( df[label], df[feature] )
    abs_coeff = abs( coeff.correlation )
    if abs_coeff > best_value:
        best_value = abs_coeff
        best_feature = feature

print("Feature %s with merit %.4f"%(best_feature, best_value))

Feature X19 with merit 0.5568


In [6]:
class PriorityQueue:
    def  __init__(self):
        self.queue = []
    def isEmpty(self):
        return len(self.queue) == 0
    def push(self, item, priority):
        for index, (i, p) in enumerate(self.queue):
            if (set(i) == set(item)):
                if (p >= priority):
                    break
                del self.queue[index]
                self.queue.append( (item, priority) )
                break
        else:
            self.queue.append( (item, priority) )
    def pop(self):
        # return item with highest priority and remove it from queue
        max_idx = 0
        for index, (i, p) in enumerate(self.queue):
            if (self.queue[max_idx][1] < p):
                max_idx = index
        (item, priority) = self.queue[max_idx]
        del self.queue[max_idx]
        return (item, priority)

In [14]:
queue = PriorityQueue()
queue.push([best_feature], best_value)

visited = []
n_backtrack = 0
max_backtrack = 5

In [15]:
while not queue.isEmpty():
    subset, priority = queue.pop()

    if (priority < best_value):
        n_backtrack += 1
    else:
        best_value = priority
        best_subset = subset
    print(subset, priority)

    if (n_backtrack == max_backtrack):
        break

    for feature in features:
        temp_subset = subset + [feature]

        for node in visited:
            if (set(node) == set(temp_subset)):
                break
        else:
            visited.append( temp_subset )
            merit = getMerit(temp_subset, label)
            queue.push(temp_subset, merit)

['X19'] 0.6257570973620136
['X19', 'X14'] 0.6151150325983153
['X19', 'X14', 'X4'] 0.6257570973620136
['X19', 'X14', 'X4', 'X7'] 0.6208308299135394
['X19', 'X14', 'X7'] 0.6039019046337366
['X19', 'X14', 'X4', 'X15'] 0.6036473315021972
['X19', 'X14', 'X4', 'X7', 'X6'] 0.5959661264680417
